In [1]:
# Установка необходимых библиотек (если запускаете в новом Colab)
!pip install earthengine-api geemap

import ee
import geemap

# 1. Авторизация и инициализация
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='gee-antonbiatov')
    # ee.Initialize(project='MY-GEE-PROJECT')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.8 MB/s eta 0:00:00


In [2]:


# 2. Определение AOI (Области интереса)
# ВАЖНО: Убедитесь, что ваш аккаунт имеет доступ к этому ассету.
try:
    pilot_aois = ee.FeatureCollection('projects/gee-antonbiatov/assets/gana_2025_galamsey/pilot_aois')
    pilot_aois_geom = pilot_aois.geometry()

    # Геометрия из границ
    aoi_base = ee.Feature(pilot_aois.bounds()).geometry()

    # Буфер -50000 метров (уменьшение области)
    aoi = aoi_base.buffer(-50000).bounds()
    print("Ассет успешно загружен.")
except Exception as e:
    print("Ошибка загрузки ассета (нет доступа или не найден). Использую тестовую точку (Гана).")
    # Резервная геометрия (Центр Ганы), если ассет недоступен
    aoi = ee.Geometry.Point([-1.0, 6.0]).buffer(10000).bounds()

# 3. Параметры
OrbitNumber = 45
start_Year = 2022
end_Year = 2025

params = {
    'aoi': aoi,
    'startYear': start_Year,
    'endYear': end_Year,
    'polarizations': ['VV', 'VH'],
    'orbit': 'ASCENDING',
    'applyAngleMask': True,

    # Спекл-фильтр (boxcar)
    'FILTER_KERNEL_SIZE': 7,
    'enableBoxcar': True,

    # Экспорт
    'exportRGB': True,           # True -> 8-бит RGB; False -> float dB бэнды
    'scale': 10,                 # м/пиксель
    'crs': 'EPSG:3857',
    'driveFolder': f'GEE_S1_monthly_{start_Year}_{end_Year}_Orbit_{OrbitNumber}',
    'prefix': '',
    'maxPixels': 1e13
}

# 4. Вспомогательные функции

def mask_by_incidence_angle(img):
    """Маска по углу падения: 30–45°"""
    angle = img.select('angle')
    mask = angle.gte(30).And(angle.lte(45))
    return img.updateMask(mask)

def boxcar(image):
    """BOXCAR SPECKLE FILTER"""
    KERNEL_SIZE = params['FILTER_KERNEL_SIZE']
    # Получаем имена каналов, исключая 'angle'
    band_names = image.bandNames().remove('angle')
    kernel = ee.Kernel.square(radius=(KERNEL_SIZE/2), units='pixels', normalize=True)

    # Применяем фильтр только к выбранным каналам
    output = image.select(band_names).convolve(kernel).rename(band_names)

    # Добавляем отфильтрованные каналы, перезаписывая старые
    return image.addBands(output, None, True)

def s1_to_rgb(img):
    """RGB из dB (R=VV, G=VH, B=VHVV)"""
    return ee.Image.rgb(
        img.select('VV').unitScale(-20, -5).multiply(255).toUint8(),
        img.select('VH').unitScale(-20, -11).multiply(255).toUint8(),
        img.select('VHVV').unitScale(-10, -5).multiply(255).toUint8()
    )

def build_s1_collection(aoi_geom, pols, orbit, apply_angle_mask):
    """Подготовка коллекции S1 с мозаицированием слайсов по dataTakeID"""

    # 1. Фильтруем коллекцию слайсов
    col = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi_geom) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.eq('resolution_meters', 10)) \
        .filter(ee.Filter.eq('productType', 'GRD')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pols[0])) \
        .filter(ee.Filter.eq('relativeOrbitNumber_start', OrbitNumber))

    if len(pols) > 1:
        col = col.filter(ee.Filter.listContains('transmitterReceiverPolarisation', pols[1]))

    if orbit != 'BOTH':
        col = col.filter(ee.Filter.eq('orbitProperties_pass', orbit))

    s1_images = col

    # --- НАЧАЛО ИНТЕГРИРОВАННОГО КОДА ---

    # 2. Список missionDataTakeID
    data_take_ids = ee.List(s1_images.aggregate_array('missionDataTakeID')).distinct()

    # 3. Функция мозаики (определена внутри, чтобы видеть s1_images)
    def mosaic_by_data_take(id_obj):
        id_num = ee.Number(id_obj)
        subset = s1_images.filter(ee.Filter.eq('missionDataTakeID', id_num))

        # Сортируем и мозаицируем
        mosaic = subset.sort('sliceNumber').mosaic()

        first = ee.Image(subset.sort('system:time_start').first())

        mosaic_out = mosaic \
            .copyProperties(first) \
            .set('missionDataTakeID', id_num) \
            .set('date', ee.Date(first.get('system:time_start'))) \
            .set('system:time_start', first.get('system:time_start')) \
            .set('sliceCount', subset.size())

        return mosaic_out

    # 4. Превращаем список ID в коллекцию мозаик
    mosaic_col = ee.ImageCollection.fromImages(
        data_take_ids.map(mosaic_by_data_take)
    ).sort('system:time_start')

    # --- КОНЕЦ ИНТЕГРИРОВАННОГО КОДА ---

    # 5. Пост-обработка
    processed_col = mosaic_col

    if apply_angle_mask:
        processed_col = processed_col.map(mask_by_incidence_angle)

    if params['enableBoxcar']:
        processed_col = processed_col.map(boxcar)

    # 6. Выбор бэндов и клип
    # В Python конкатенация списков делается через +
    bands_to_select = pols + ['angle']

    return processed_col.select(bands_to_select).map(lambda img: img.clip(aoi_geom))

def make_monthly_windows(start_date, end_date):
    """Генерация списка месячных диапазонов"""
    n_months = end_date.difference(start_date, 'month').toInt()

    def create_window(m):
        start = start_date.advance(m, 'month')
        end = start.advance(1, 'month')
        return ee.Dictionary({
            'start': start,
            'end': end,
            'start_str': start.format('YYYY-MM-dd'),
            'end_str': end.format('YYYY-MM-dd'),
            'label': start.format('YYYY-MM')
        })

    return ee.List.sequence(0, n_months.subtract(1)).map(create_window)

def build_monthly_image(start_d, end_d, pols):
    """Создание месячного композита (последний снимок)"""
    month_col = s1.filterDate(start_d, end_d)

    # Берем самый новый снимок
    latest_img = month_col \
        .sort('system:time_start', False) \
        .first() \
        .select(pols)

    # Добавляем VHVV
    if 'VV' in pols and 'VH' in pols:
        vhvv = latest_img.select('VH').subtract(latest_img.select('VV')).rename('VHVV')
        latest_img = latest_img.addBands(vhvv)

    if params['exportRGB']:
        return s1_to_rgb(latest_img)
    else:
        return latest_img

# 5. ОСНОВНОЙ КОД ВЫПОЛНЕНИЯ

start_date = ee.Date.fromYMD(params['startYear'], 1, 1)
end_date = ee.Date.fromYMD(params['endYear'], 12, 31).advance(1, 'day')

print("Сбор коллекции S1...")
s1 = build_s1_collection(params['aoi'], params['polarizations'], params['orbit'], params['applyAngleMask'])

count = s1.size().getInfo()
print(f'Всего снимков Sentinel-1 после фильтров: {count}')


# 6. Подготовка и запуск экспорта
months = make_monthly_windows(start_date, end_date)

# Фильтрация пустых месяцев (Server-side logic transferred to list)
def filter_empty_months(d):
    d = ee.Dictionary(d)
    s = ee.Date(d.get('start'))
    e = ee.Date(d.get('end'))
    n = s1.filterDate(s, e).size()
    return ee.Algorithms.If(n.gt(0), d, None)

available_months = months.map(filter_empty_months).removeAll([None])

# Получаем список месяцев на клиент (Python)
month_dicts = available_months.getInfo()

if params['exportRGB'] and len(params['polarizations']) < 2:
    print('Внимание: exportRGB=True, но не хватает поляризаций для RGB.')

print(f"Найдено месяцев для экспорта: {len(month_dicts)}")

# Запуск задач экспорта
tasks = []

for md in month_dicts:
    label = md['label']
    s = ee.Date(md['start_str'])
    e = ee.Date(md['end_str'])

    # Генерируем изображение
    img = build_monthly_image(s, e, params['polarizations']) \
        .set('label', label) \
        .set('system:time_start', s.millis())

    desc = f"{params['prefix']}_{label}" if params['prefix'] else label

    export_args = {
        'image': img,
        'description': desc,
        'folder': params['driveFolder'],
        'fileNamePrefix': desc,
        'region': params['aoi'],
        'scale': params['scale'],
        'maxPixels': params['maxPixels'],
        'fileFormat': 'GeoTIFF',
        'formatOptions': {'cloudOptimized': True}
    }

    if params['crs']:
        export_args['crs'] = params['crs']

    # Создаем и запускаем задачу
    task = ee.batch.Export.image.toDrive(**export_args)
    task.start()
    tasks.append(label)
    print(f"Задача запущена: {desc}")

print('Все задачи поставлены в очередь.')

Ассет успешно загружен.
Сбор коллекции S1...
Всего снимков Sentinel-1 после фильтров: 461
Найдено месяцев для экспорта: 47
Задача запущена: 2022-01
Задача запущена: 2022-02
Задача запущена: 2022-03
Задача запущена: 2022-04
Задача запущена: 2022-05
Задача запущена: 2022-06
Задача запущена: 2022-07
Задача запущена: 2022-08
Задача запущена: 2022-09
Задача запущена: 2022-10
Задача запущена: 2022-11
Задача запущена: 2022-12
Задача запущена: 2023-01
Задача запущена: 2023-02
Задача запущена: 2023-03
Задача запущена: 2023-04
Задача запущена: 2023-05
Задача запущена: 2023-06
Задача запущена: 2023-07
Задача запущена: 2023-08
Задача запущена: 2023-09
Задача запущена: 2023-10
Задача запущена: 2023-11
Задача запущена: 2023-12
Задача запущена: 2024-01
Задача запущена: 2024-02
Задача запущена: 2024-03
Задача запущена: 2024-04
Задача запущена: 2024-05
Задача запущена: 2024-06
Задача запущена: 2024-07
Задача запущена: 2024-08
Задача запущена: 2024-09
Задача запущена: 2024-10
Задача запущена: 2024-11
За

In [3]:

# Визуализация на карте (опционально)
m = geemap.Map()
m.centerObject(params['aoi'], 9)
m.addLayer(params['aoi'], {'color': 'red'}, 'AOI')
# Пример добавления первого снимка для проверки
if count > 0:
    first_img = s1.first()
    vis_params = {'min': -25, 'max': 0} if not params['exportRGB'] else {}
    # Для RGB визуализация уже "вшита" в значения 0-255, поэтому параметры не нужны
    m.addLayer(first_img.select(['VV', 'VH']), {'min':-25, 'max':0}, 'First Image Raw', False)
m # Отобразить карту в ячейке

Map(center=[5.6851720650249415, -1.8449998759757766], controls=(WidgetControl(options=['position', 'transparen…